In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pickle

lang_1 = pickle.load(open('data_beer_lemm_90K.p', 'rb'))
# with open('data_beer_embed_90K.p', 'rb') as f:
#     u = pickle._Unpickler(f)
#     u.encoding = 'latin1'
#     lang_1 = u.load()

In [3]:
data_train = lang_1['training']

In [4]:
lang_1['vocab_size'] = len(lang_1['word2idx'])
lang_1['word_dim'] = 200

In [8]:
import torch
import model
mod = model.Model(lang_1=lang_1, num_aspect=20)

In [9]:
import random    
import time

import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def train(model, epochs, iters, batch_size=64) :
    start = time.time()
    
    for epoch in range(1, epochs) :
        loss_total = 0
        loss_reg_total = 0
        
        random.shuffle(data_train)
        
        for i in range(0, iters, batch_size):
            torch.cuda.empty_cache()
            batch_start = i
            batch_end = batch_start + batch_size
            input_sent = data_train[batch_start:batch_end]
            
            bsize = len(input_sent)
            neg_sents = []
            for i in range(20) :
                neg_start = random.randint(0, len(data_train) - batch_size)
                neg_end = neg_start + bsize
                neg_sents += [data_train[neg_start:neg_end]]
                
            model.train_batch(input_sent, neg_sents, do_tanh=True)
            loss_total += model.loss
            loss_reg_total += model.loss_reg
        
        dirname = model.save_values("_beer_tanh")
            
        loss = loss_total / (iters / batch_size)
        loss_reg = loss_reg_total / (iters / batch_size)
        
        progress = '%s (%d %d%%)' % (timeSince(start, epoch/epochs), epoch, epoch/epochs * 100)
        output = progress + '\t' + str(loss) + '\t' + str(loss_reg)
        f = open(dirname + '/epochs.txt', 'a')
        f.write(output + '\n')
        f.close()
        print(output)

In [ ]:
iters = len(data_train) // 100
print(iters)
train(mod, 10, len(data_train), 100)

8390
9m 11s (- 82m 43s) (1 10%)	4.320076088901157	1.0236940761585644


In [50]:
import torch.nn.functional as F
topv, topi = torch.mm(F.normalize(mod.model.embedding.weight, dim=-1), 
                      F.normalize(mod.model.T.weight, dim=0)).topk(20, dim=0)

In [51]:
aspect = []
for i in range(30) :
    aspect.append([lang_1['idx2word'][y] for y in topi.data[:, i]])

In [ ]:
mod.model.embedding.weight.size()

In [52]:
import pandas as pd
pd.DataFrame(aspect)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,preeclamptic,placental,asphyxiated,homocysteine,deformability,thirteen,twin,fetus,fourteen,seventeen,hemoglobin,twelve,singleton,twenty,subnormal,eighteen,sixteen,gestational,normoalbuminuric,placenta
1,markedly,lower,significantly,reduced,increased,decreased,substantially,dramatically,occurrence,drastically,whereas,slightly,higher,consistently,observed,elevated,considerably,diminished,due,weaker
2,qqq,boy,girl,retarded,isrctn,taylor,woman,amdp,daughter,stunted,queen,lose,cry,tpn,husband,conceive,cd,baby,teenager,anomaly
3,curative,undergoing,surgery,surgical,ablative,conserving,debulking,abdominoperineal,resection,oncological,laparoscopic,advancement,mastectomy,oncologic,fifty,seventy,operable,undergone,forty,sixty
4,misoprostol,mifepristone,danazol,tocolytic,thalidomide,tocolytics,methotrexate,ciclosporin,valacyclovir,mesalazine,isotretinoin,prophylaxis,acyclovir,contraceptive,sulphasalazine,ketoconazole,metronidazole,prednisolone,azathioprine,sulfasalazine
5,qqq,chulalongkorn,debris,foam,pigment,deposit,glomerulus,mucus,endothelium,mesangial,endothelin,endothelial,role,melanin,lining,technician,smooth,plaque,dilated,calcification
6,barbiturate,isoflurane,anesthetic,sevoflurane,neonate,halothane,effect,anaesthetic,anesthesia,prenatal,anaesthesia,cocaine,adrenaline,fetus,intrathecal,etomidate,maternal,doxapram,epidural,dexmedetomidine
7,hundred,eighty,sixty,forty,fourteen,seventy,fourty,thirty,fifty,nineteen,seventeen,ninety,twenty,thirteen,sixteen,eighteen,tracheobronchial,fifteen,consecutive,bronchopneumonia
8,serious,adverse,barotrauma,preventable,unintended,morbidity,outcome,unacceptable,spirometric,arterial,modifiable,airflow,minimising,thromboembolic,cardiovascular,objectively,adequacy,compromise,compliance,stillbirth
9,pfannenstiel,surgeon,suturing,shoulder,perineal,epidural,compression,analgesia,eversion,abdominal,intraoperatively,ligasure,perineum,thoracic,extraperitoneal,operative,lichtenstein,suprapubic,obturator,intraoperative
